In [1]:
#이게 진짜 분산 서버 중 리뷰 서버임!!
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/review_server

/content/drive/MyDrive/review_server


In [ ]:
!pip install bertopic
!pip install kss
!pip install flask flask-cors pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
!python review_server.py

2025-06-10 16:34:08.679262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749573248.920790    3168 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749573248.989861    3168 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 16:34:09.564722: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
✅ 리뷰 분석 서버 ngrok URL: https://3adc-34-143-249-81.ngrok-free.app
 * Serving Flask app 'review_server'
 * Debug mode: o

In [ ]:
import os
import sys

# 기본 경로 설정
BASE_DIR = '/content/drive/MyDrive/review_server'

# sys.path에 기본 경로 추가
sys.path.append(BASE_DIR)
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import os, re, json
from review_analysis import run_topic_model_on_room  # 리뷰 분석 함수

# ngrok 설정
ngrok.set_auth_token("2yGSKnM6Tviku0bqCV7bRN5y7gn_rLmTrz5SsPvRgd62yS5b")

app = Flask(__name__)
CORS(app)

# 결과 저장 폴더
RESULT_FOLDER = './results'
os.makedirs(RESULT_FOLDER, exist_ok=True)

# ngrok 연결
public_url = ngrok.connect(5000).public_url
print(f"✅ 리뷰 분석 서버 ngrok URL: {public_url}")

# ✅ 리뷰 분석만 수행하는 엔드포인트
@app.route('/analyze_review', methods=['POST'])
def analyze_review():
    data = request.get_json()
    url = data.get('url')
    print("🔍 리뷰 분석 요청받은 URL:", url)

    if not url:
        return jsonify({"status": "error", "message": "숙소 URL이 없습니다."}), 400

    try:
        # 숙소 ID 추출 (파일명으로도 사용)
        room_id_match = re.search(r'/rooms/(\d+)', url)
        if not room_id_match:
            return jsonify({"status": "error", "message": "유효한 숙소 URL이 아닙니다."}), 400
        room_id = room_id_match.group(1)

        # 리뷰 분석 실행
        result = run_topic_model_on_room(url)

        # 결과를 JSON 파일로 저장 (선택적)
        save_path = os.path.join(RESULT_FOLDER, f"{room_id}.json")
        with open(save_path, "w", encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False)

        # 최종 결과 JSON으로 반환
        return jsonify({
            "status": "success",
            "room_id": room_id,
            "result": result
        })

    except Exception as e:
        print(f"[!] 리뷰 분석 실패: {e}")
        return jsonify({"status": "error", "message": str(e)}), 500


# Flask 실행
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


✅ 리뷰 분석 서버 ngrok URL: https://fa28-34-126-64-9.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 03:19:17] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 03:19:17] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 03:19:17] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 03:19:17] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 03:23:09] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 03:25:10] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 03:38:37] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 03:38:55] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 03:38:55] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 03:38:56] "GE